MOVIE RECOMMENDATION SYSTEM 

In [20]:
# movie recommmendation system
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

df_m=pd.read_csv("tmdb_5000_movies.csv")
#df_m = pd.read_csv("3D Objects/tmdb_5000_movies.csv")



#creating connection with sql server db
from sqlalchemy import create_engine
from sqlalchemy.sql import text
engine = create_engine('mssql+pyodbc://@DESKTOP-AC0JV2J\\SQLEXPRESS/mov_recmd?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes')

print("enter your user_id if have else create your user_id ")
a= int(input("press 1 to login else 2 to create you id "))

if a == 1 :
    #checking if user id exist
    
    while True:
        # Take input from the user
        idd = input("Enter your 7 digit ID: ")

        # Ensure the input is valid (7 digits)
        if not idd.isdigit() or len(idd) != 7:
            print("Please enter a valid 7-digit ID.")
            continue

        # Query to check if the ID exists in the database
        query = text("SELECT id FROM views_history")

        # Execute the query
        with engine.connect() as connection:
            result = connection.execute(query)
        
            # Check if the entered ID exists in the query result
            id_exists = False
            for row in result:
                if str(row[0]) == idd:  # Compare the string representation of both
                    id_exists = True
                    print("yes your ID already exists in the database.")
                    break

        # If ID exists, break the loop and proceed
        if id_exists:
            break
        else:
            print("ID does not exist. Please try again.")
            
    #show him the movie according to his past and tf-idf method 
    #fetch name of movie he watched recently 





    engine = create_engine('mssql+pyodbc://@DESKTOP-AC0JV2J\\SQLEXPRESS/mov_recmd?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes')

    

    # Define the query with a placeholder for the ID
    query = text("SELECT TOP 1 movie_name FROM views_history WHERE id = :idd ORDER BY date_time DESC")

    # Execute the query with the idd parameter
    with engine.connect() as connection:
        result = connection.execute(query, {'idd': idd})
    
        # Fetch the result
        row = result.fetchone()

        # Check if a result was returned
        if row:
            # Access the first column, which is the movie_name
            movie_watched_last_day = row[0]
            print(f"Latest movie watched: {row[0]}")
        else:
            print("No movie found for the given ID.")
    # now feed him movies list by tf-idf method
    a=TfidfVectorizer(stop_words="english")
    df_m["overview"]= df_m["overview"].fillna('')
    tfidf_matrix=a.fit_transform(df_m["overview"])
    cos_relation=linear_kernel(tfidf_matrix,tfidf_matrix)
    indices=pd.Series(df_m.index, index=df_m["original_title"]).drop_duplicates() 
    
    def get_name (title, cosine_sim= cos_relation):
        idx=indices[title]
        sim_scores= enumerate (cos_relation[idx])
        sim_scores=sorted(sim_scores, key=lambda x : x[1],reverse=True)
        sim_scores=sim_scores[1:21]
    
        sim_index=[i[0] for i in sim_scores]
        print("these are the recommended movies for you")
        print(df_m["original_title"].iloc[sim_index])
    get_name(movie_watched_last_day)
elif a == 2:
    
    
    while True:
        # Take input from the user
        idd = input("Enter a 7-digit int: ")

        # Ensure the input is valid (7 digits)
        if not idd.isdigit() or len(idd) != 7:
            print("Please enter a valid 7-digit ID.")
            continue

        # Query to check if the ID exists in the database
        query = text("SELECT id FROM views_history WHERE id = :idd")

        # Execute the query
        with engine.connect() as connection:
            result = connection.execute(query, {"idd": idd}).fetchone()
        
            # Check if the entered ID exists in the query result
            if result:
                print("This ID already exists in the database. Please try again.")
            else:
                print("OK, THIS ID IS GOOD TO BE CREATED.")
            
                # Insert the new ID and commit the transaction
                insert_query = text("INSERT INTO views_history (id) VALUES (:idd)")
                connection.execute(insert_query, {"idd": idd})
            
                # Commit the transaction explicitly
                connection.commit()  # Commit the transaction
                print(f"ID {idd} SUCCESSFULLY CREATED.")
            
                break  # Exit the loop after successful insertion
    
    #from a particular genre 
    # now we want to get 100 movies with a particular genre and latest relasing date 
    # we should note that the genere column in dataframe is a list of many genres
    for index, row in df_m.iterrows():
        current_genre = row['genres']

    
        'converting this str to list to extract the genere'
        import ast

        actual_list = ast.literal_eval(current_genre)

        #print(actual_list)
        # Extract the 'name' from each dictionary, convert to lowercase, and create a list of names
        l2 = [genre['name'].lower() for genre in actual_list]

    
    
       # Put the transformed data back to the respective cell
    
        df_m.at[index, 'genres'] = ', '.join(l2)


    
    unique_genres = set()

    #Iterate over each row's genre string
    for genres_string in df_m['genres']:
        #Clean and split the genres string
        genres_list = [genre.strip().strip('"') for genre in genres_string.split(',')]
    
        # Add cleaned genres to the set
        unique_genres.update(genres_list)

     #Convert the set back to a list
    l3 = list(unique_genres)

    print("\nDistinct genres:")
    print(l3)
    #len(l3)
    #here we get l3 as the list of all genre available to us 
    # take input from user 
    user_fav_genre = input("enter your prefered genre from the list ")
    #print(user_fav_genre)

    if user_fav_genre in l3 :
        print("okk")
    else:
        print("error")
    data_types = df_m.dtypes

    # convert release date to datetime
    df_m['release_date'] = pd.to_datetime(df_m['release_date'])

    #find top 200 movies released recently 
    top_movies = df_m.sort_values(by='release_date', ascending=False).head(2000)
    #print(top_movies.dtypes)


    # Filter rows where the genre contains the word 'Action'
    action_movies = top_movies[top_movies['genres'].str.contains(user_fav_genre, case=False, na=False)]

    print(action_movies['original_title'])
 
    # a df from highest ratings 
    # now we are getting top 2000 rated movies 
    top_rating_m = df_m.sort_values(by='vote_average', ascending=False).head(2000)
    top_rating_m['original_title']
    
    # a df from most  views counts 
    trending_100_movies = df_m.sort_values(by='popularity', ascending=False).head(1000)
    trending_100_movie_names = trending_100_movies['original_title']
    #print(trending_100_movie_names)

    # now we will combine these three lists and then recmnd his a final list
    #finding common movies of all three df
    a=top_rating_m
    b=action_movies
    c=trending_100_movies
    common_elements = set(a.index) & set(b.index) & set(c.index)
    common_elements =list(common_elements)
    print(df_m['original_title'].loc[common_elements])
    
    #user will select a movie from this 
    while True:
        m = input("Select a movie name from this: ")

        # Convert both to lowercase for case-insensitive matching
        movie_list = df_m['original_title'].loc[common_elements].str.lower()

        if m.lower() in movie_list.values:
            print("OK, enjoy your movie!")
            break  # Exit the loop when a correct movie name is entered
        else:
            print("Something went wrong. Please check the spelling and try again.")
    
    
    # store that (id , m_name , time) in db
    
    query = text("insert into views_history(id, movie_name, date_time) values (:idd, :movie_name, getdate())")


    with engine.connect() as connection:
        # Start a transaction
        trans = connection.begin()
        try:
            # Execute the insert query
            connection.execute(query, {'idd': idd, 'movie_name': m})
        
            # Commit the transaction
            trans.commit()
            print(f"The movie '{m}' has been successfully stored in the database.")
        except Exception as e:
            # Rollback the transaction if there is an error
            trans.rollback()
            print(f"Error occurred: {e}")
else :
    print("error")
        


    

enter your user_id if have else create your user_id 


press 1 to login else 2 to create you id  1
Enter your 7 digit ID:  6767677


yes your ID already exists in the database.
Latest movie watched: drive


KeyError: 'drive'